# Trabalhando com tabelas e dados geográficos

Para essa parte, você deverá baixar os dados no [Google Drive](https://drive.google.com/file/d/1BoaLcy0Ew_rSUVqroYwKvFMZDpWgAVUI/view?usp=sharing) e colocá-los na seguinte estrutura:

    data -> colocar aqui dados das UBS e o IBGE de Maringá
    data/municipios -> colocar aqui os dados dos municipios do PR
    data/setores -> colocar aqui os dados dos setores de Maringá

In [ ]:
using DataFrames, CSV, Query
using GeoDataFrames, ArchGDAL
using Plots

In [ ]:
gr()

In [ ]:
municipios_estado = GeoDataFrames.read("data/municipios/PR_Municipios_2022.shp")

In [ ]:
n_cidades = nrow(municipios_estado)

In [ ]:
plot(municipios_estado.geometry, aspectratio=true, grid=false, alpha=0.5, color_palette=palette(:Paired, n_cidades))
plot!(GeoDataFrames.boundary.(municipios_estado.geometry), color=:black, aspectratio=true, grid=false)

### Qual a cidade com maior área?

In [ ]:
municipios_estado |> 
    @filter(_.AREA_KM2 == maximum(municipios_estado.AREA_KM2)) |> 
    @select(:NM_MUN, :AREA_KM2) |> 
    DataFrame

In [ ]:
plot(municipios_estado.geometry, column="AREA_KM2", aspectratio=true, grid=false, color_palette=:Reds)
#plot!(plt, GeoDataFrames.boundary.(municipios_estado.geometry), color=:white)
#plt

In [ ]:
maringa = municipios_estado |> @filter(_.NM_MUN == "Maringá") |> @select(-:SIGLA_UF, -:AREA_KM2) |> DataFrame

In [ ]:
cod_maringa = 4115200

In [ ]:
plot(GeoDataFrames.boundary.(maringa.geometry), color=:black, aspectratio=true, grid=false)

### Que outra pergunta podemos fazer?

## Utilizando as malhas de setores censitários

Agora iremos utilizar uma divisão feita pelo Censo para separar áreas dentro de municípios. Esses dados podem ser obtidos [aqui](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/malhas-territoriais/26565-malhas-de-setores-censitarios-divisoes-intramunicipais.html).

Para esta análise, utilizaremos os dados do tipo _Shape File_ apenas do estado do Paraná.

In [ ]:
setores = GeoDataFrames.read("data/setores/PR_Malha_Preliminar_2022.shp");
first(setores, 10)

In [ ]:
println(names(setores))

Ao filtar pelo código do município de Maringá, observamos que nada vem. Isso mostra que precisamos saber o **tipo** de dados que é lido. Neste caso, o código, ao invés de ser lido como um número é lido como uma *String* (texto), então utilizamos o comando `string` para converter o número em texto e filtrar.

In [ ]:
setores_maringa = setores |> @filter(_.CD_MUN == cod_maringa) |> DataFrame

In [ ]:
setores_maringa = setores |> @filter(_.CD_MUN == string(cod_maringa)) |> @select(:AREA_KM2, :NM_MUN, :CD_SETOR, :geometry) |> DataFrame;
first(setores_maringa, 5)

In [ ]:
# plot(GeoDataFrames.boundary.(maringa.geometry), color=:black, aspectratio=true, grid=false)
plot(GeoDataFrames.boundary.(setores_maringa.geometry), color=:black, aspectratio=true, grid=false)


## Unidades Básicas de Saúde

Agora utilizaremos os dados obtidos em [Dados.gov.br](https://dados.gov.br/) sobre as posições da Unidades Básicas de Saúde de todo o Brasil, filtrando-as pelo código de Maringá e desenhando-as no mapa.

Para essa tarefa, teremos que:

  1. Baixar os dados
  1. Carregar as informações
  1. Identificar as colunas desejadas e filtrar pela cidade de Maringá
  1. Converter as latitudes e longitudes em dados de mapa
  1. Desenhar

In [ ]:
ubs_brasil = CSV.read("data/ubs.csv.gz", DataFrame)

In [ ]:
names(ubs_brasil)

Temos 2 problemas com esses dados:

  1. Como encontrar Maringá?
  1. Como extrair a latitude e longitude?

In [ ]:
ubs_maringa = ubs_brasil |> @filter(_.IBGE == cod_maringa / 10) |> DataFrame

In [ ]:
ubs_maringa[!, "geometry"] = ArchGDAL.createpoint.(parse.(Float64, replace.(convert.(String, ubs_maringa.LONGITUDE), "," => ".")), parse.(Float64, replace.(convert.(String, ubs_maringa.LATITUDE), "," => ".")))

In [ ]:
plot(GeoDataFrames.boundary.(setores_maringa.geometry), color=:red, aspectratio=true, grid=false)
plot!(ubs_maringa.geometry, marker=:circle, color=:green, markersize=3)
#xaxis!((-52.0, -51.9))
#yaxis!((-23.50, -23.35))

## Os dados do IBGE

Esses dados vieram do Censo 2022 e, pela primeira vez, o IBGE colocou informações de georreferenciamento, removendo dados sensíveis dos entrevistados. Aqui, só iremos usar para contar as pessoas e desenhar.

In [ ]:
ibge_maringa = CSV.read("data/IBGE_maringa.csv.gz", DataFrame);
first(ibge_maringa, 5)

In [ ]:
size(ibge_maringa)

In [ ]:
ibge_maringa[!, "geometry"] = ArchGDAL.createpoint.(ibge_maringa.LONGITUDE, ibge_maringa.LATITUDE)

In [ ]:
populacao_setorial = ibge_maringa |> @groupby(_.COD_SETOR) |> @map({CD_SETOR=convert(String, key(_)), populacao=length(_)}) |> DataFrame

In [ ]:
densidade = setores_maringa |> @join(populacao_setorial, _.CD_SETOR, _.CD_SETOR, {_.CD_SETOR, _.geometry, _.AREA_KM2, __.populacao}) |> DataFrame
densidade[!, "densidade"] = densidade.populacao ./ densidade.AREA_KM2

In [ ]:
plot(densidade.geometry, column="densidade", color_palette=:Reds, aspectratio=true, grid=false)
xaxis!((-52.0, -51.9))
yaxis!((-23.50, -23.35))

## Convertendo coordenadas

Geralmente, as coordenadas são dadas em *ângulos*, mas calcular a distância entre dois pontos dados por ângulos não é recomendada, pois é necessário utilizar um modelo elipsoidal da Terra, uma distância comumente chamada de distância geodésica. Para resolver isso, nós projetamos o nosso mapa usando alguns dos muitos esquemas de representação do nosso mundo em coordenadas, chamados de ESPG.

No nosso caso, nossos dados são dados no modelos utilizado pelo IBGE 4674 e utilizaremos um sistema projetivo dado pelo ESPG 5880, adequado para a América Latina (utilizado pela Petrobrás, inclusive). Para mais informações sobre o ESPG, acesse o [site deles](https://epsg.io/).

In [ ]:
using GeoFormatTypes

In [ ]:
c_origem = GeoFormatTypes.EPSG(4674);
c_destino = GeoFormatTypes.EPSG(5880);

In [ ]:
ubs = ubs_maringa |> @select(:CNES, :NOME, :geometry) |> DataFrame;

In [ ]:
reproject(ubs[:, "geometry"], c_origem, c_destino, order=:trad)
reproject(maringa.geometry, c_origem, c_destino, order=:trad)

In [ ]:
plot(GeoDataFrames.boundary.(maringa.geometry), color=:black, aspectratio=true, grid=false)
plot!(ubs_maringa.geometry, marker=:circle, color=:green, markersize=3)

Precisamos voltar aos valores originais, por questões de funcionamento interno desse pacote.

In [ ]:
reproject(ubs[:, "geometry"], c_destino, c_origem, order=:trad)
reproject(maringa.geometry, c_destino, c_origem, order=:trad)